In [2]:
# Reference: https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Copy data to google colab from google drive and unzip
# This may take 1-2 minutes
!cp gdrive/My\ Drive/ADBI\ Project/Dataset/ner_dataset.csv.zip .
!unzip ner_dataset.csv.zip

Archive:  ner_dataset.csv.zip
replace ner_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


## Implementation

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

### Load Data

In [6]:
df = pd.read_csv('ner_dataset.csv', encoding = "ISO-8859-1")
df = df[:10000]
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


## Data Preprocessing

In [9]:
print(df.isnull().sum())
df = df.fillna(method='ffill')
print(df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique())
df.groupby('Tag').size().reset_index(name='counts')
print("Pre processing done")

Sentence #    0
Word          0
POS           0
Tag           0
dtype: int64
457 2746 17
Pre processing done


###Train Test split - 70-30%

In [10]:
X = df.drop('Tag', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
print(X_train.shape, y_train.shape)
new_classes = classes.copy()
new_classes.pop()

(6700, 3242) (6700,)


'O'

## CRF - Conditional Random Fields 

In [11]:

! pip install sklearn_crfsuite

In [0]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

### Sentences

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

### Feature Extraction and Train-Test split

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features
  
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

### CRF model

In [0]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)
y_pred = crf.predict(X_test)

print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

              precision    recall  f1-score   support

       B-art       0.50      0.40      0.44         5
       B-eve       0.00      0.00      0.00         2
       B-geo       0.79      0.68      0.73        77
       B-gpe       0.75      0.88      0.81        91
       B-nat       0.00      0.00      0.00         2
       B-org       0.77      0.68      0.72        53
       B-per       0.85      0.92      0.88        61
       B-tim       0.95      0.89      0.92        45
       I-art       0.00      0.00      0.00         4
       I-eve       0.00      0.00      0.00         1
       I-geo       0.75      0.38      0.50        16
       I-gpe       0.67      0.57      0.62         7
       I-nat       0.00      0.00      0.00         2
       I-org       0.74      0.70      0.72        50
       I-per       0.87      0.97      0.92        75
       I-tim       0.33      1.00      0.50         1

   micro avg       0.80      0.78      0.79       492
   macro avg       0.50   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
# Transition
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(10))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-10:])

# Check the state features 
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(10))
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-10:])


Top likely transitions:
B-org  -> I-org   4.680201
B-per  -> I-per   4.242035
I-org  -> I-org   4.198692
I-per  -> I-per   3.991878
B-art  -> I-art   3.688363
B-eve  -> I-eve   3.636531
I-art  -> I-art   3.464745
B-gpe  -> I-gpe   3.365085
O      -> O       3.119727
B-geo  -> I-geo   2.966060

Top unlikely transitions:
B-tim  -> B-gpe   -0.627473
B-geo  -> I-art   -0.651209
B-org  -> I-per   -0.664598
B-geo  -> I-per   -0.781181
B-gpe  -> I-per   -0.828000
I-org  -> I-per   -0.837846
O      -> I-per   -0.970128
O      -> I-art   -1.289684
O      -> I-tim   -1.493870
O      -> I-org   -1.870850
Top positive:
5.860922 O        bias
3.722280 O        BOS
3.721897 B-tim    word[-3:]:day
3.690997 B-tim    word[-2:]:ay
3.149401 B-gpe    word.istitle()
3.047903 B-tim    word[-2:]:0s
2.978401 B-geo    -1:word.lower():in
2.502835 O        postag:NN
2.437516 B-tim    -1:word.lower():in
2.386781 B-gpe    postag:JJ

Top negative:
-1.410707 B-gpe    -1:word.lower():from
-1.434462 B-gpe    word[-3:]

### ELi5

In [0]:
! pip install eli5
import eli5
eli5.show_weights(crf, top=10)
# eli5.show_weights(crf, top=10, targets=['O', 'B-org', 'I-per'])
# eli5.show_weights(crf, top=10, feature_re='^word\.is',horizontal_layout=False, show=['targets'])

    100% |████████████████████████████████| 102kB 2.8MB/s 


From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,3.12,1.111,-1.29,0.688,-0.58,1.332,-0.605,1.572,-0.537,0.0,1.103,-1.871,0.028,-0.97,1.764,-1.494
B-art,0.0,0.0,3.688,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.011,0.0,-0.216,0.0,0.0
I-art,-0.058,0.0,3.465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.229,0.0,0.0
B-eve,-0.506,0.0,0.0,0.0,3.637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.074,0.0,0.0
I-eve,0.0,0.0,0.0,0.0,1.834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-geo,0.197,0.0,-0.651,0.0,-0.284,-0.182,2.966,0.687,-0.471,0.0,-0.17,-0.613,-0.224,-0.781,1.057,-0.217
I-geo,-0.246,0.0,0.0,0.0,0.0,0.0,1.734,0.0,0.0,0.0,0.0,0.0,0.0,-0.041,0.0,0.0
B-gpe,0.435,0.0,-0.39,0.0,-0.14,-0.206,-0.244,-0.624,3.365,0.0,0.785,-0.49,1.263,-0.828,-0.271,0.0
I-gpe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.934,0.0,0.0,0.0,0.0,-0.093,0.0,0.0
B-nat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### CRF Trained model
Using the CRF model to predict the tagging for a given  text.

In [0]:

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

